In [1]:
REF_EXPERIMENT = '0810_ADAM_0.1_l2_lr_0.00001_20210810-091159'

In [2]:
!pip install icecream
from icecream import ic

import os
import json
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
from google.colab import drive

MOUNT_POINT = '/content/gdrive' 
drive.mount(MOUNT_POINT,  force_remount=True)
MOUNT_POINT += "/My Drive"

Mounted at /content/gdrive


In [4]:
#folders
PWD = os.path.join(MOUNT_POINT, "4.2/visiope/PROJECT")

In [5]:
#export result
EXPORT_ROOT = os.path.join(PWD, 'results')
if not os.path.isdir(EXPORT_ROOT):
  os.makedirs(EXPORT_ROOT)
EXPORT_CURRENT_EXPERIMENT = os.path.join(EXPORT_ROOT, REF_EXPERIMENT)

In [6]:
SETUPS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'setups')
MODELS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'models')
LOG_TRAINING_TESTING = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'log_training_testing')
BEST_WEIGHTS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'weights')

LOGS = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'logs')

CHECKPOINT = os.path.join(EXPORT_CURRENT_EXPERIMENT, 'checkpoint')

In [7]:
def export_setting(name_setting, setting):
  #export setting
  filename = os.path.join(SETUPS, name_setting)
  with open(filename, "w") as outfile:
      json.dump(setting, outfile, indent=4)
      outfile.close()

def load_setting(name_setting):
    #import setting
    filename = os.path.join(SETUPS, name_setting)
    setting = {}
    with open(filename, "r") as infile:
        setting=json.load(infile)
        infile.close()
    return setting

In [8]:
SETTING = load_setting('SETTING.json')
SETTING

{'ADAM_LEARNING_RATE': 1e-05,
 'BATCH_SIZE': 10,
 'DATASET': 'dataset_aug',
 'EPOCHS': 50,
 'IMG_HEIGHT': 135,
 'IMG_WIDTH': 240,
 'LOSS': 'binary_cross_entropy',
 'MODEL': 'RRUNET',
 'OPTIMIZER': 'adam',
 'SEED_RANDOM': 20201205,
 'SGD_LEARNING_RATE': 0.1,
 'SGD_MOMENTUM': 0.9,
 'SGD_NESTEROV': True,
 'SGD_WEIGHT_DECAY': 0.1}

In [9]:
#input dataset
DATASET_ROOT_IMAGE = os.path.join(PWD, SETTING['DATASET'], 'images')

## TRAIN
TRAINING_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'train/images')

## VALIDATION
VALIDATION_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'val/images')

## TEST
TEST_SET_IMAGE = os.path.join(DATASET_ROOT_IMAGE, 'test/images')

In [10]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 27.1 MB/s 


In [11]:
import numpy as np
import tensorflow as tf

import keras

print("Tensorflow version %s" %tf.__version__)
print("Keras version %s" %keras.__version__)

device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
  #raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.7.0
Keras version 2.7.0
Found GPU at: 


In [12]:
#set seeds
from numpy.random import seed 
seed(SETTING['SEED_RANDOM'])

tf.random.set_seed(SETTING['SEED_RANDOM'])

In [13]:
!pip install tensorflow-addons
!pip install tensorflow-io

     |████████████████████████████████| 22.7 MB 1.5 MB/s 


In [14]:
import sys
sys.path.append(PWD)

import RRU_net
import Dataset
import Utils

# TESTING

## Dataset

In [15]:
# TEST AND VALIDATION SETS
dataset_builder = Dataset.Dataset(training_image_folder=TEST_SET_IMAGE,
                                  validation_image_folder=VALIDATION_SET_IMAGE, 
                                  batch_size=SETTING['BATCH_SIZE'], 
                                  random_seed=SETTING['SEED_RANDOM'],
                                  img_height=SETTING['IMG_HEIGHT'],
                                  img_width=SETTING['IMG_WIDTH'])

None
Tensor("case/cond/Identity:0", shape=(), dtype=string)
None
Tensor("case/cond/Identity:0", shape=(), dtype=string)


In [16]:
dataset = dataset_builder.dataset

TEST_SIZE = dataset_builder.training_set_size
VALSET_SIZE = dataset_builder.validation_set_size

In [17]:
print(dataset['train'])
print(dataset['val'])

<PrefetchDataset shapes: ((None, 135, 240, None), (None, 135, 240, 1)), types: (tf.float32, tf.uint8)>
<PrefetchDataset shapes: ((None, 135, 240, None), (None, 135, 240, 1)), types: (tf.float32, tf.uint8)>


## Model

In [18]:
model = RRU_net.RRU_net(l2_penalty=SETTING['SGD_WEIGHT_DECAY'])
input_x = tf.random.uniform((1, 6, 14, 3))
y_hat = model(input_x)
model.summary()

Model: "rru_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 down_0 (RRU_first_down)     multiple                  10528     
                                                                 
 down_1 (RRU_down)           multiple                  59904     
                                                                 
 down_2 (RRU_down)           multiple                  238592    
                                                                 
 down_3 (RRU_down)           multiple                  952320    
                                                                 
 down_4 (RRU_down)           multiple                  1312768   
                                                                 
 up_1 (RRU_up)               multiple                  1132288   
                                                                 
 up_2 (RRU_up)               multiple                  2835

### Load weights

In [19]:
def get_best_weights():
  for filename in sorted(os.listdir(BEST_WEIGHTS), reverse=True):
    if 'f1_score' in filename:
      return os.path.join(BEST_WEIGHTS, filename)

In [20]:
best_weights = get_best_weights()
best_weights

'/content/gdrive/My Drive/4.2/visiope/PROJECT/results/0810_ADAM_0.1_l2_lr_0.00001_20210810-091159/weights/f1_score=0.3301944236650173__epoch=21'

In [21]:
Utils.load_weights(model, best_weights)

## Evaluation

In [22]:
def evaluation(model, tf_dataset, dataset_size, batch_size) -> dict:
  tf_recall = tf.keras.metrics.Recall()
  tf_precision = tf.keras.metrics.Precision()
  
  for step in range(dataset_size // batch_size):
    for image, mask in tf_dataset.take(1):
      y_pred = model.predict(image)
      y_pred = tf.reshape(y_pred, [-1])

      y_true = tf.reshape(mask, [-1])

      tf_recall.update_state(y_true, y_pred)
      tf_precision.update_state(y_true, y_pred)
  
  recall = tf_recall.result().numpy()
  precision = tf_precision.result().numpy()
  f1_score = 2*(precision*recall)/(precision+recall)

  return {'recall':recall, 'precision': precision, 'f1_score': f1_score}


def eval_val():
  return evaluation(model, dataset['val'], VALSET_SIZE, SETTING['BATCH_SIZE'])
def eval_test():
  return evaluation(model, dataset['train'], TEST_SIZE, SETTING['BATCH_SIZE'])

In [23]:
eval_val()

{'f1_score': 0.3274001141163096, 'precision': 0.29297635, 'recall': 0.37099028}

In [24]:
eval_test()

{'f1_score': 0.2295993019567551, 'precision': 0.2672746, 'recall': 0.20123328}

In [25]:
def create_mask(prediction: tf.Tensor):
  threshold = 0.5
  res = tf.math.greater(prediction, threshold)
  res = tf.cast(res, tf.uint8)*255
  return res

def mask_to_PIL_image(mask: tf.Tensor):
  img = tf.keras.preprocessing.image.array_to_img(mask)
  return img

def prediction_to_PIL_image_mask(prediction: tf.Tensor):
  mask = create_mask(prediction)
  return mask_to_PIL_image(mask)

In [26]:
def display_sample(display_list):
    """Show side-by-side an input image,
    the ground truth and the prediction.
    """
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

def show_predictions(dataset=None, num=1):
  for images, masks in dataset.take(num):
      pred_masks = model.predict(images)

      for i in range(len(images)):      
        pred_mask = create_mask(pred_masks[i])
        true_mask = masks[i]
        display_sample([images[i], true_mask, pred_mask])



In [27]:
show_predictions(dataset['val'], num=10) # VALIDATION SET

Output hidden; open in https://colab.research.google.com to view.

In [28]:
show_predictions(dataset['train'], num=10) # TEST SET

Output hidden; open in https://colab.research.google.com to view.